In [14]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [15]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [17]:
# simple 3 layer NN
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [18]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


In [22]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [24]:
flatten = nn.Flatten()
flat_image = flatten(input_image) # to 1d array
print(flat_image.size())

torch.Size([3, 784])


In [25]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [26]:
# rectified linear unit, x for positive, 0 for negative
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0328, -0.2911, -0.1678,  0.0968,  0.2591, -0.2240,  0.2719,  0.6105,
         -0.0192, -0.0854,  0.6021, -0.2773, -0.0336, -0.1442,  0.1078,  0.4651,
          0.2323, -0.3492,  0.0414,  0.4090],
        [ 0.2740, -0.1545,  0.3211,  0.4389, -0.0068, -0.0616, -0.0065,  0.5061,
          0.1360,  0.1327,  0.3780, -0.2926, -0.3972, -0.4353, -0.2608,  0.3239,
          0.1655, -0.2454,  0.1947,  0.3745],
        [ 0.7181, -0.2612,  0.1434,  0.5464,  0.0177,  0.0212,  0.3542,  0.6563,
          0.1646,  0.0748,  0.7309, -0.0858,  0.0233, -0.1402, -0.2473,  0.1254,
          0.0915, -0.0505,  0.0872,  0.4753]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0968, 0.2591, 0.0000, 0.2719, 0.6105, 0.0000,
         0.0000, 0.6021, 0.0000, 0.0000, 0.0000, 0.1078, 0.4651, 0.2323, 0.0000,
         0.0414, 0.4090],
        [0.2740, 0.0000, 0.3211, 0.4389, 0.0000, 0.0000, 0.0000, 0.5061, 0.1360,
         0.1327, 0.3780, 0.0000, 0.0000, 0.0000, 0.00

In [28]:
# simple sequential of 28x28 -> 20 -> 10
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [29]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [30]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0298, -0.0171, -0.0177,  ..., -0.0220,  0.0236, -0.0154],
        [-0.0093,  0.0157, -0.0130,  ..., -0.0276, -0.0263,  0.0249]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0063,  0.0294], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0363,  0.0055,  0.0348,  ...,  0.0433, -0.0089,  0.0029],
        [-0.0099, -0.0225, -0.0052,  ..., -0.0297,  0.0341, -0.0406]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 